In [12]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Load JSON data
json_dir = '/content'
all_games = []
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        try:
            with open(os.path.join(json_dir, filename), 'r', encoding='utf-8') as f:
                all_games.append(json.load(f))
        except Exception as e:
            print(f"Failed to load {filename}: {e}")

In [14]:
# Extract match-level features
matches = []
for game in all_games:
    info = game.get('info', {})
    innings = game.get('innings', [])
    teams = info.get('teams', [None, None])
    toss_winner = info.get('toss', {}).get('winner')
    toss_decision = info.get('toss', {}).get('decision')
    venue = info.get('venue')
    winner = info.get('outcome', {}).get('winner')
    first_batting = innings[0]['team'] if innings else None
    if winner is None or winner == 'no result':
        continue
    matches.append({
        'team1': teams[0],
        'team2': teams[1],
        'toss_winner': toss_winner,
        'toss_decision': toss_decision,
        'venue': venue,
        'first_batting': first_batting,
        'winner': winner
    })
match_df = pd.DataFrame(matches)
match_df.to_csv('bigbash_match_features.csv', index=False)

In [15]:
# Extract ball-by-ball deliveries
deliveries = []
for game in all_games:
    info = game.get('info', {})
    innings = game.get('innings', [])
    match_id = info.get('event', {}).get('match_number') or info.get('dates', [None])[0]
    for inning in innings:
        batting_team = inning.get('team')
        for over in inning.get('overs', []):
            over_number = over.get('over')
            for delivery in over.get('deliveries', []):
                runs = delivery.get('runs', {})
                extras = runs.get('extras', 0)
                wicket = 1 if 'wickets' in delivery else 0
                deliveries.append({
                    'match_id': match_id,
                    'batting_team': batting_team,
                    'over': over_number,
                    'batter': delivery.get('batter'),
                    'bowler': delivery.get('bowler'),
                    'total_runs': runs.get('total', 0),
                    'extras': extras,
                    'wicket': wicket
                })
ball_df = pd.DataFrame(deliveries)
ball_df.to_csv('bigbash_ball_by_ball.csv', index=False)

In [16]:
# Extract detailed stats
matches_stats = []
batting_stats = []
bowling_stats = []
extras_stats = []
wickets_stats = []

for game in all_games:
    info = game.get('info', {})
    innings = game.get('innings', [])
    event = info.get('event', {})
    match_id = event.get('match_number') or info.get('dates', [None])[0]
    season = info.get('season')
    venue = info.get('venue')
    teams = info.get('teams', [None, None])
    toss = info.get('toss', {})
    toss_winner = toss.get('winner')
    toss_decision = toss.get('decision')
    outcome = info.get('outcome', {})
    winner = outcome.get('winner')
    if winner is None or winner == 'no result':
        continue
    matches_stats.append({
        'match_id': match_id,
        'season': season,
        'venue': venue,
        'team1': teams[0],
        'team2': teams[1],
        'toss_winner': toss_winner,
        'toss_decision': toss_decision,
        'winner': winner
    })
    for inning_idx, inning in enumerate(innings, start=1):
        batting_team = inning.get('team')
        wides = sum(delivery.get('extras', {}).get('wides', 0) for over in inning.get('overs', []) for delivery in over.get('deliveries', []))
        noballs = sum(delivery.get('extras', {}).get('noballs', 0) for over in inning.get('overs', []) for delivery in over.get('deliveries', []))
        byes = sum(delivery.get('extras', {}).get('byes', 0) for over in inning.get('overs', []) for delivery in over.get('deliveries', []))
        legbyes = sum(delivery.get('extras', {}).get('legbyes', 0) for over in inning.get('overs', []) for delivery in over.get('deliveries', []))
        extras_stats.append({
            'match_id': match_id,
            'inning': inning_idx,
            'batting_team': batting_team,
            'wides': wides,
            'noballs': noballs,
            'byes': byes,
            'legbyes': legbyes
        })
        batsmen_stats = {}
        for over in inning.get('overs', []):
            for delivery in over.get('deliveries', []):
                batter = delivery.get('batter')
                runs_bat = delivery.get('runs', {}).get('batter', 0)
                extras = delivery.get('extras', {})
                stats = batsmen_stats.setdefault(batter, {'runs':0, 'balls':0, 'fours':0, 'sixes':0})
                stats['runs'] += runs_bat
                if 'wides' not in extras and 'noballs' not in extras:
                    stats['balls'] += 1
                if runs_bat == 4:
                    stats['fours'] += 1
                elif runs_bat == 6:
                    stats['sixes'] += 1
        for player, stats in batsmen_stats.items():
            balls = stats['balls']
            runs = stats['runs']
            strike_rate = round((runs / balls) * 100, 2) if balls > 0 else 0
            batting_stats.append({
                'match_id': match_id,
                'inning': inning_idx,
                'batting_team': batting_team,
                'player': player,
                'runs_scored': runs,
                'balls_faced': balls,
                'strike_rate': strike_rate,
                'fours': stats['fours'],
                'sixes': stats['sixes']
            })
        bowlers_stats = {}
        for over in inning.get('overs', []):
            for delivery in over.get('deliveries', []):
                bowler = delivery.get('bowler')
                runs_total = delivery.get('runs', {}).get('total', 0)
                extras = delivery.get('extras', {})
                wickets_list = delivery.get('wickets', [])
                stats = bowlers_stats.setdefault(bowler, {'balls':0, 'runs_conceded':0, 'wickets':0, 'extras_conceded':0})
                if 'wides' not in extras and 'noballs' not in extras:
                    stats['balls'] += 1
                stats['runs_conceded'] += runs_total
                stats['extras_conceded'] += sum(extras.values()) if extras else 0
                stats['wickets'] += len(wickets_list)
        for bowler, stats in bowlers_stats.items():
            overs_bowled = stats['balls'] // 6 + (stats['balls'] % 6) / 6
            economy = round(stats['runs_conceded'] / overs_bowled, 2) if overs_bowled > 0 else 0
            bowling_stats.append({
                'match_id': match_id,
                'inning': inning_idx,
                'bowling_team': teams[0] if batting_team == teams[1] else teams[1],
                'player': bowler,
                'overs_bowled': round(overs_bowled, 2),
                'maidens': 0,
                'runs_conceded': stats['runs_conceded'],
                'wickets': stats['wickets'],
                'extras_conceded': stats['extras_conceded'],
                'economy': economy
            })
        for over in inning.get('overs', []):
            for delivery in over.get('deliveries', []):
                for wicket in delivery.get('wickets', []):
                    kind = wicket.get('kind')
                    player_out = wicket.get('player_out')
                    fielders = [f.get('name') for f in wicket.get('fielders', []) if 'name' in f]
                    wickets_stats.append({
                        'match_id': match_id,
                        'inning': inning_idx,
                        'batting_team': batting_team,
                        'player_out': player_out,
                        'kind': kind,
                        'fielders': ', '.join(fielders)
                    })

pd.DataFrame(matches_stats).to_csv('bb_match_info.csv', index=False)
pd.DataFrame(batting_stats).to_csv('bb_batting_stats.csv', index=False)
pd.DataFrame(bowling_stats).to_csv('bb_bowling_stats.csv', index=False)
pd.DataFrame(extras_stats).to_csv('bb_extras_stats.csv', index=False)
pd.DataFrame(wickets_stats).to_csv('bb_wickets_stats.csv', index=False)

In [17]:
# Define BigBashWinnerPredictor class (omitting intermediate detailed docstrings for brevity)
class BigBashWinnerPredictor:

    def __init__(self):
        self.player_batting_features = {}
        self.player_bowling_features = {}
        self.team_features = {}
        self.venue_features = {}
        self.scalers = {}
        self.label_encoders = {}
        self.models = {}
        self.ensemble_model = None
        self.feature_names = []
        self.is_trained = False
        self.model_performance = {}

    def calculate_player_batting_features(self, batting_data):
        player_stats = batting_data.groupby('player').agg({
            'runs_scored': ['mean', 'std', 'sum', 'count', 'max'],
            'balls_faced': ['mean', 'std', 'sum'],
            'strike_rate': ['mean', 'std', 'max'],
            'fours': ['mean', 'sum'],
            'sixes': ['mean', 'sum']
        }).fillna(0)
        player_stats.columns = ['_'.join(col).strip() for col in player_stats.columns]
        for player in player_stats.index:
            player_data = batting_data[batting_data['player'] == player]
            player_stats.loc[player, 'batting_consistency'] = 1 / (1 + player_stats.loc[player, 'runs_scored_std'])
            high_scores = (player_data['runs_scored'] >= 30).sum()
            player_stats.loc[player, 'high_score_frequency'] = high_scores / len(player_data)
            total_runs = player_stats.loc[player, 'runs_scored_sum']
            boundary_runs = player_stats.loc[player, 'fours_sum'] * 4 + player_stats.loc[player, 'sixes_sum'] * 6
            player_stats.loc[player, 'boundary_percentage'] = (boundary_runs / max(total_runs, 1)) * 100
            recent_innings = player_data.tail(5)['runs_scored'].mean()
            player_stats.loc[player, 'recent_batting_form'] = recent_innings
            player_stats.loc[player, 'sr_consistency'] = 1 / (1 + player_stats.loc[player, 'strike_rate_std'])
        return player_stats

    def calculate_player_bowling_features(self, bowling_data):
        player_stats = bowling_data.groupby('player').agg({
            'overs_bowled': ['sum', 'mean', 'count'],
            'runs_conceded': ['sum', 'mean', 'std'],
            'wickets': ['sum', 'mean', 'max'],
            'economy': ['mean', 'std', 'min'],
            'extras_conceded': ['sum', 'mean']
        }).fillna(0)
        player_stats.columns = ['_'.join(col).strip() for col in player_stats.columns]
        for player in player_stats.index:
            player_data = bowling_data[bowling_data['player'] == player]
            total_runs = player_stats.loc[player, 'runs_conceded_sum']
            total_wickets = player_stats.loc[player, 'wickets_sum']
            player_stats.loc[player, 'bowling_average'] = total_runs / max(total_wickets, 1)
            total_balls = player_stats.loc[player, 'overs_bowled_sum'] * 6
            player_stats.loc[player, 'bowling_strike_rate'] = total_balls / max(total_wickets, 1)
            player_stats.loc[player, 'wicket_percentage'] = (player_data['wickets'] > 0).sum() / len(player_data)
            player_stats.loc[player, 'economy_consistency'] = 1 / (1 + player_stats.loc[player, 'economy_std'])
            recent_wickets = player_data.tail(5)['wickets'].mean()
            recent_economy = player_data.tail(5)['economy'].mean()
            player_stats.loc[player, 'recent_bowling_form'] = recent_wickets / max(recent_economy, 1)
            player_stats.loc[player, 'dot_ball_rate'] = max(0, (6 - player_stats.loc[player, 'economy_mean']) / 6)
        return player_stats

    def calculate_team_features(self, match_data, batting_data, bowling_data, wickets_data):
        teams = pd.concat([match_data['team1'], match_data['team2']]).unique()
        team_stats = {}
        for team in teams:
            team_matches = match_data[(match_data['team1'] == team) | (match_data['team2'] == team)]
            wins = (match_data['winner'] == team).sum()
            toss_wins = (match_data['toss_winner'] == team).sum()
            team_batting = batting_data[batting_data['batting_team'] == team]
            team_bowling = bowling_data[bowling_data['bowling_team'] == team]
            team_wickets_lost = wickets_data[wickets_data['batting_team'] == team]
            team_stats[team] = {
                'win_percentage': wins / len(team_matches) if len(team_matches) else 0,
                'total_matches': len(team_matches),
                'toss_win_percentage': toss_wins / len(team_matches) if len(team_matches) else 0,
                'avg_team_runs': team_batting.groupby('match_id')['runs_scored'].sum().mean() if len(team_batting) else 0,
                'avg_team_strike_rate': team_batting['strike_rate'].mean() if len(team_batting) else 0,
                'runs_consistency': 1 / (1 + team_batting.groupby('match_id')['runs_scored'].sum().std()) if len(team_batting) else 0,
                'avg_team_economy': team_bowling['economy'].mean() if len(team_bowling) else 0,
                'avg_team_wickets': team_bowling.groupby('match_id')['wickets'].sum().mean() if len(team_bowling) else 0,
                'bowling_consistency': 1 / (1 + team_bowling['economy'].std()) if len(team_bowling) else 0,
                'avg_wickets_lost': team_wickets_lost.groupby('match_id').size().mean() if len(team_wickets_lost) else 0
            }
        return team_stats

    def calculate_venue_features(self, match_data, batting_data):
        venue_stats = {}
        venues = match_data['venue'].unique()
        for venue in venues:
            venue_matches = match_data[match_data['venue'] == venue]
            venue_match_ids = venue_matches['match_id'].tolist()
            venue_batting = batting_data[batting_data['match_id'].isin(venue_match_ids)]
            if len(venue_batting):
                match_totals = venue_batting.groupby('match_id')['runs_scored'].sum()
                venue_stats[venue] = {
                    'avg_runs_per_match': match_totals.mean(),
                    'avg_strike_rate': venue_batting['strike_rate'].mean(),
                    'high_score_frequency': (venue_batting['runs_scored'] >= 30).sum() / len(venue_batting),
                    'six_frequency': venue_batting['sixes'].sum() / len(venue_batting),
                    'four_frequency': venue_batting['fours'].sum() / len(venue_batting),
                    'total_matches': len(venue_matches),
                    'runs_variance': match_totals.std(),
                    'bat_first_preference': (venue_matches['toss_decision'] == 'bat').sum() / len(venue_matches) if len(venue_matches) else 0.5,
                    'home_advantage': 0
                }
            else:
                venue_stats[venue] = {
                    'avg_runs_per_match': 0, 'avg_strike_rate': 0, 'high_score_frequency': 0,
                    'six_frequency': 0, 'four_frequency': 0, 'total_matches': len(venue_matches),
                    'runs_variance': 0, 'bat_first_preference': 0.5, 'home_advantage': 0
                }
        return venue_stats

    def create_match_features(self, team1, team2, venue, team1_players=None, team2_players=None):
        features = []
        feature_names = []
        if team1 in self.team_features and team2 in self.team_features:
            team1_stats = self.team_features[team1]
            team2_stats = self.team_features[team2]
            features.extend([
                team1_stats['win_percentage'] - team2_stats['win_percentage'],
                team1_stats['avg_team_runs'] - team2_stats['avg_team_runs'],
                team2_stats['avg_team_economy'] - team1_stats['avg_team_economy'],
                team1_stats['avg_team_strike_rate'] - team2_stats['avg_team_strike_rate']
            ])
            feature_names.extend([
                'win_percentage_diff', 'batting_strength_diff', 'bowling_strength_diff', 'strike_rate_diff'
            ])
            for team_name, team_stats in [(team1, team1_stats), (team2, team2_stats)]:
                prefix = 'team1' if team_name == team1 else 'team2'
                features.extend([
                    team_stats['win_percentage'], team_stats['toss_win_percentage'], team_stats['avg_team_runs'],
                    team_stats['avg_team_strike_rate'], team_stats['avg_team_economy'], team_stats['avg_team_wickets'],
                    team_stats['avg_wickets_lost']
                ])
                feature_names.extend([
                    f'{prefix}_win_percentage', f'{prefix}_toss_win_percentage', f'{prefix}_avg_runs',
                    f'{prefix}_avg_strike_rate', f'{prefix}_avg_economy', f'{prefix}_avg_wickets', f'{prefix}_avg_wickets_lost'
                ])
        else:
            features.extend([0]*18)
            feature_names.extend([
                'win_percentage_diff', 'batting_strength_diff', 'bowling_strength_diff', 'strike_rate_diff',
                'team1_win_percentage', 'team1_toss_win_percentage', 'team1_avg_runs', 'team1_avg_strike_rate',
                'team1_avg_economy', 'team1_avg_wickets', 'team1_avg_wickets_lost',
                'team2_win_percentage', 'team2_toss_win_percentage', 'team2_avg_runs', 'team2_avg_strike_rate',
                'team2_avg_economy', 'team2_avg_wickets', 'team2_avg_wickets_lost'
            ])
        if venue in self.venue_features:
            v = self.venue_features[venue]
            features.extend([v['avg_runs_per_match'], v['avg_strike_rate'], v['high_score_frequency'], v['six_frequency'], v['bat_first_preference']])
        else:
            features.extend([0] * 5)
        feature_names.extend(['venue_avg_runs', 'venue_avg_strike_rate', 'venue_high_score_freq', 'venue_six_freq', 'venue_bat_first_pref'])
        if team1_players and team2_players and self.player_batting_features is not None:
            team1_batting_total = sum(self.player_batting_features.loc[p, 'runs_scored_mean'] for p in team1_players if p in self.player_batting_features.index)
            team1_bowling_total = sum(self.player_bowling_features.loc[p, 'wickets_mean'] for p in team1_players if p in self.player_bowling_features.index)
            team1_batting_count = sum(1 for p in team1_players if p in self.player_batting_features.index)
            team1_bowling_count = sum(1 for p in team1_players if p in self.player_bowling_features.index)
            team2_batting_total = sum(self.player_batting_features.loc[p, 'runs_scored_mean'] for p in team2_players if p in self.player_batting_features.index)
            team2_bowling_total = sum(self.player_bowling_features.loc[p, 'wickets_mean'] for p in team2_players if p in self.player_bowling_features.index)
            team2_batting_count = sum(1 for p in team2_players if p in self.player_batting_features.index)
            team2_bowling_count = sum(1 for p in team2_players if p in self.player_bowling_features.index)
            team1_batting_avg = team1_batting_total / max(team1_batting_count, 1)
            team1_bowling_avg = team1_bowling_total / max(team1_bowling_count, 1)
            team2_batting_avg = team2_batting_total / max(team2_batting_count, 1)
            team2_bowling_avg = team2_bowling_total / max(team2_bowling_count, 1)
            features.extend([
                team1_batting_avg - team2_batting_avg, team1_bowling_avg - team2_bowling_avg,
                team1_batting_avg, team1_bowling_avg, team2_batting_avg, team2_bowling_avg
            ])
        else:
            features.extend([0] * 6)
        feature_names.extend([
            'batting_strength_player_diff', 'bowling_strength_player_diff',
            'team1_batting_player_avg', 'team1_bowling_player_avg',
            'team2_batting_player_avg', 'team2_bowling_player_avg'
        ])
        while len(features) < 29:
            features.append(0)
        return np.array(features), feature_names

    def prepare_training_data(self, match_data, batting_data, bowling_data, wickets_data):
        self.player_batting_features = self.calculate_player_batting_features(batting_data)
        self.player_bowling_features = self.calculate_player_bowling_features(bowling_data)
        self.team_features = self.calculate_team_features(match_data, batting_data, bowling_data, wickets_data)
        self.venue_features = self.calculate_venue_features(match_data, batting_data)
        X, y = [], []
        for _, match in match_data.iterrows():
            match_id = match['match_id']
            team1, team2 = match['team1'], match['team2']
            venue = match['venue']
            winner = match['winner']
            match_batting = batting_data[batting_data['match_id'] == match_id]
            team1_players = match_batting[match_batting['batting_team'] == team1]['player'].unique().tolist()
            team2_players = match_batting[match_batting['batting_team'] == team2]['player'].unique().tolist()
            features, feature_names = self.create_match_features(team1, team2, venue, team1_players, team2_players)
            if len(features) == 29:
                X.append(features)
                y.append(winner)
        self.feature_names = feature_names
        return np.array(X), np.array(y), feature_names

    def train_models(self, X, y):
        X = np.nan_to_num(X, nan=0.0, posinf=1e6, neginf=-1e6)
        self.scalers['standard'] = StandardScaler()
        X_scaled = self.scalers['standard'].fit_transform(X)
        self.label_encoders['winner'] = LabelEncoder()
        y_encoded = self.label_encoders['winner'].fit_transform(y)
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_encoded, test_size=0.2, random_state=42,
            stratify=y_encoded if len(np.unique(y_encoded)) > 1 else None
        )
        models = {
            'RandomForest': RandomForestClassifier(
                n_estimators=300, max_depth=12, min_samples_split=5, min_samples_leaf=2,
                max_features='sqrt', random_state=42, class_weight='balanced', n_jobs=-1),
            'GradientBoosting': GradientBoostingClassifier(
                n_estimators=200, learning_rate=0.1, max_depth=8, min_samples_split=10,
                min_samples_leaf=4, subsample=0.8, random_state=42),
            'LogisticRegression': LogisticRegression(
                random_state=42, class_weight='balanced', max_iter=1000, C=1.0, solver='liblinear'),
            'SVM': SVC(
                probability=True, random_state=42, class_weight='balanced', C=1.0, gamma='scale', kernel='rbf'),
            'NeuralNetwork': MLPClassifier(
                hidden_layer_sizes=(128, 64, 32), random_state=42, max_iter=1000,
                learning_rate='adaptive', early_stopping=True, validation_fraction=0.1)
        }
        model_scores = {}
        for name, model in models.items():
            model.fit(X_train, y_train)
            cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
            test_score = model.score(X_test, y_test)
            model_scores[name] = {'cv_mean': cv_scores.mean(), 'cv_std': cv_scores.std(), 'test_score': test_score, 'model': model}
            self.models[name] = model
        ensemble_estimators = [(n, m) for n, m in models.items() if n != 'NeuralNetwork']
        self.ensemble_model = VotingClassifier(estimators=ensemble_estimators, voting='soft')
        self.ensemble_model.fit(X_train, y_train)
        ensemble_score = self.ensemble_model.score(X_test, y_test)
        model_scores['Ensemble'] = {'test_score': ensemble_score}
        self.model_performance = model_scores
        self.is_trained = True
        return X_test, y_test, model_scores

    def predict_winner(self, team1, team2, venue, team1_players=None, team2_players=None):
        if not self.is_trained:
            return {'error': 'Model not trained. Call train_from_csv_files() first.'}
        features, _ = self.create_match_features(team1, team2, venue, team1_players, team2_players)
        if len(features) == 0:
            return {'error': 'Unable to create features - insufficient data'}
        features = np.nan_to_num(features, nan=0.0, posinf=1e6, neginf=-1e6)
        features_scaled = self.scalers['standard'].transform([features])
        predictions = {}
        probabilities = {}
        for name, model in self.models.items():
            pred = model.predict(features_scaled)[0]
            pred_proba = model.predict_proba(features_scaled)[0]
            predicted_team = self.label_encoders['winner'].inverse_transform([pred])[0]
            predictions[name] = predicted_team
            probabilities[name] = {team: pred_proba[i] for i, team in enumerate(self.label_encoders['winner'].classes_)}
        ensemble_pred = self.ensemble_model.predict(features_scaled)[0]
        ensemble_proba = self.ensemble_model.predict_proba(features_scaled)[0]
        ensemble_predicted_team = self.label_encoders['winner'].inverse_transform([ensemble_pred])[0]
        ensemble_probabilities = {team: ensemble_proba[i] for i, team in enumerate(self.label_encoders['winner'].classes_)}
        max_prob = max(ensemble_probabilities.values())
        confidence_level = 'High' if max_prob > 0.7 else 'Medium' if max_prob > 0.5 else 'Low'
        return {
            'prediction': ensemble_predicted_team,
            'confidence': max_prob,
            'confidence_level': confidence_level,
            'probabilities': ensemble_probabilities,
            'individual_predictions': predictions,
            'individual_probabilities': probabilities,
            'match_details': {
                'team1': team1,
                'team2': team2,
                'venue': venue,
                'players_included': bool(team1_players and team2_players)
            }
        }

    def train_from_csv_files(self, batting_file='bb_batting_stats.csv', bowling_file='bb_bowling_stats.csv',
                             match_file='bb_match_info.csv', wickets_file='bb_wickets_stats.csv'):
        try:
            batting_data = pd.read_csv(batting_file)
            bowling_data = pd.read_csv(bowling_file)
            match_data = pd.read_csv(match_file)
            wickets_data = pd.read_csv(wickets_file)
        except FileNotFoundError as e:
            print(f"Error loading files: {e}")
            return False
        X, y, _ = self.prepare_training_data(match_data, batting_data, bowling_data, wickets_data)
        self.train_models(X, y)
        return True

    def get_feature_importance(self, top_n=15):
        if not self.is_trained or 'RandomForest' not in self.models:
            return None
        rf_model = self.models['RandomForest']
        if hasattr(rf_model, 'feature_importances_'):
            importances = rf_model.feature_importances_
            return sorted(zip(self.feature_names, importances), key=lambda x: x[1], reverse=True)[:top_n]
        return None

    def model_summary(self):
        if not self.is_trained:
            print("Model not trained yet.")
            return
        for name, scores in self.model_performance.items():
            if name != 'Ensemble':
                print(f"{name:20}: {scores['test_score']:.3f} (CV: {scores.get('cv_mean', 0):.3f})")
            else:
                print(f"{name:20}: {scores['test_score']:.3f}")
        importance = self.get_feature_importance()
        if importance:
            print("\nTop predictive features:")
            for i, (feature, imp) in enumerate(importance[:10]):
                print(f"  {i+1:2d}. {feature:25}: {imp:.3f}")

    def save_model(self, filepath='bb_predictor_model.pkl'):
        if self.is_trained:
            joblib.dump(self, filepath)
            print(f"Model saved to {filepath}")
        else:
            print("No trained model to save")

    @staticmethod
    def load_model(filepath='bb_predictor_model.pkl'):
        try:
            return joblib.load(filepath)
        except FileNotFoundError:
            print(f"Model file {filepath} not found")
            return None

In [18]:
# Define team squads
squads = {
    'Adelaide Strikers': ["CJ Boyce", "JS Buckingham", "AT Carey", "T Kelly", "HJ Nielsen", "J Overton", "L Pope", "AI Ross", "JJS Sangha", "MW Short", "HTRY Thornton"],
    'Brisbane Heat': ["XC Bartlett", "M Bryant", "Lachlan Hearne", "Spencer Johnson", "UT Khawaja", "MP Kuhnemann", "NA McSweeney", "C Munro", "MG Neser", "JJ Peirson", "MT Renshaw", "Callum Vidler", "HD Weibgen", "JD Wildermuth"],
    'Hobart Hurricanes': ["Iain Carlisle", "Nikhil Chaudhary", "NT Ellis", "CJ Jordan", "BR McDermott", "Riley Meredith", "Mitch Owen", "MS Wade", "J Weatherald", "BJ Webster", "Mac Wright"],
    'Melbourne Renegades': ["JP Behrendorff", "Josh Brown", "Harry Dixon", "B Doggett", "J Fraser-McGurk", "CP Jewell", "NM Lyon", "F O'Neill", "TS Rogers", "TL Seifert", "W Sutherland", "A Zampa"],
    'Melbourne Stars': ["SM Boland", "Hilton Cartwright", "TK Curran", "Liam Hatcher", "GJ Maxwell", "Hamish McKenzie", "TS Rogers", "PM Siddle", "MT Steketee", "MP Stoinis", "MJ Swepson"],
    'Perth Scorchers': ["AC Agar", "FA Allen", "M Beardman", "C Connolly", "S Fanning", "AM Hardie", "NR Hobson", "JP Inglis", "Bryce Jackson", "M Kelly", "MR Marsh", "LR Morris", "JS Paris", "JA Richardson", "AJ Turner"],
    'Sydney Sixers': ["SA Abbott", "Babar Azam", "JA Davies", "BJ Dwarshuis", "J Edwards", "MC Henriques", "T Murphy", "Mitch Perry", "JR Philippe", "JC Silk", "SPD Smith"],
    'Sydney Thunder': ["TD Andrews", "WA Agar", "CT Bancroft", "SW Billings", "O Davies", "M Gilkes", "CJ Green", "S Konstas", "N McAndrew", "DR Sams", "T Sangha", "DA Warner"]
}

# Matches info
matches_info = [
    ("Dec 14", "Perth Scorchers", "Sydney Sixers", "Perth Stadium"),
    ("Dec 15", "Melbourne Renegades", "Brisbane Heat", "Simonds Stadium, South Geelong, Victoria"),
    ("Dec 16", "Hobart Hurricanes", "Sydney Thunder", "Bellerive Oval, Hobart"),
    ("Dec 17", "Sydney Sixers", "Adelaide Strikers", "Sydney Cricket Ground"),
    ("Dec 18", "Melbourne Stars", "Hobart Hurricanes", "Melbourne Cricket Ground"),
    ("Dec 19", "Brisbane Heat", "Perth Scorchers", "Brisbane Cricket Ground"),
    ("Dec 20", "Sydney Thunder", "Sydney Sixers", "Sydney Showground Stadium"),
    ("Dec 21", "Melbourne Renegades", "Hobart Hurricanes", "Simonds Stadium, South Geelong, Victoria"),
    ("Dec 22", "Sydney Thunder", "Brisbane Heat", "Manuka Oval, Canberra"),
    ("Dec 23", "Adelaide Strikers", "Melbourne Stars", "Adelaide Oval"),
    ("Dec 26", "Sydney Sixers", "Melbourne Stars", "Sydney Cricket Ground"),
    ("Dec 26", "Perth Scorchers", "Hobart Hurricanes", "Perth Stadium"),
    ("Dec 27", "Brisbane Heat", "Adelaide Strikers", "Brisbane Cricket Ground"),
    ("Dec 28", "Melbourne Stars", "Sydney Thunder", "Manuka Oval, Canberra"),
    ("Dec 29", "Hobart Hurricanes", "Melbourne Renegades", "Bellerive Oval, Hobart"),
    ("Dec 30", "Sydney Thunder", "Perth Scorchers", "Sydney Showground Stadium"),
    ("Dec 31", "Adelaide Strikers", "Brisbane Heat", "Adelaide Oval"),
    ("Jan 01", "Melbourne Renegades", "Sydney Sixers", "Docklands Stadium, Melbourne"),
    ("Jan 01", "Hobart Hurricanes", "Perth Scorchers", "Bellerive Oval, Hobart"),
    ("Jan 02", "Brisbane Heat", "Melbourne Stars", "Brisbane Cricket Ground"),
    ("Jan 03", "Sydney Thunder", "Hobart Hurricanes", "Sydney Showground Stadium"),
    ("Jan 04", "Melbourne Stars", "Melbourne Renegades", "Melbourne Cricket Ground"),
    ("Jan 04", "Perth Scorchers", "Adelaide Strikers", "Perth Stadium"),
    ("Jan 05", "Sydney Sixers", "Brisbane Heat", "International Sports Stadium, Coffs Harbour"),
    ("Jan 06", "Adelaide Strikers", "Sydney Thunder", "Adelaide Oval"),
    ("Jan 07", "Perth Scorchers", "Melbourne Renegades", "Perth Stadium"),
    ("Jan 08", "Melbourne Stars", "Sydney Sixers", "Melbourne Cricket Ground"),
    ("Jan 09", "Hobart Hurricanes", "Adelaide Strikers", "Bellerive Oval, Hobart"),
    ("Jan 10", "Brisbane Heat", "Sydney Thunder", "Brisbane Cricket Ground"),
    ("Jan 10", "Melbourne Renegades", "Melbourne Stars", "Docklands Stadium, Melbourne"),
    ("Jan 11", "Sydney Sixers", "Hobart Hurricanes", "Sydney Cricket Ground"),
    ("Jan 11", "Adelaide Strikers", "Perth Scorchers", "Adelaide Oval"),
    ("Jan 12", "Sydney Thunder", "Melbourne Renegades", "Sydney Showground Stadium"),
    ("Jan 13", "Melbourne Stars", "Adelaide Strikers", "Melbourne Cricket Ground"),
    ("Jan 14", "Hobart Hurricanes", "Brisbane Heat", "Bellerive Oval, Hobart"),
    ("Jan 15", "Melbourne Renegades", "Perth Scorchers", "Docklands Stadium, Melbourne"),
    ("Jan 16", "Sydney Sixers", "Sydney Thunder", "Sydney Cricket Ground"),
    ("Jan 17", "Adelaide Strikers", "Melbourne Renegades", "Adelaide Oval"),
    ("Jan 17", "Perth Scorchers", "Melbourne Stars", "Perth Stadium"),
    ("Jan 18", "Brisbane Heat", "Sydney Sixers", "Brisbane Cricket Ground")
]

In [23]:
# Initialize and train the predictor
predictor = BigBashWinnerPredictor()
predictor.train_from_csv_files()

predictions = []
for date, team1, team2, venue in matches_info:
    team1_players = squads.get(team1)
    team2_players = squads.get(team2)
    result_team1_bats = predictor.predict_winner(team1, team2, venue, team1_players, team2_players)
    result_team2_bats = predictor.predict_winner(team2, team1, venue, team2_players, team1_players)
    team1_bats_confidence = result_team1_bats.get('confidence', 0)
    team2_bats_confidence = result_team2_bats.get('confidence', 0)
    avg_conf_team1 = (team1_bats_confidence + result_team2_bats.get('probabilities', {}).get(team1, 0)) / 2
    avg_conf_team2 = (result_team1_bats.get('probabilities', {}).get(team2, 0) + team2_bats_confidence) / 2
    overall_winner = team1 if avg_conf_team1 > avg_conf_team2 else team2
    overall_confidence = max(avg_conf_team1, avg_conf_team2)
    predictions.append({
        'date': date,
        'team1': team1,
        'team2': team2,
        'venue': venue,
        'team1_bats_first': (result_team1_bats['prediction'], team1_bats_confidence),
        'team2_bats_first': (result_team2_bats['prediction'], team2_bats_confidence),
        'overall': (overall_winner, overall_confidence)
    })

match = 1
for p in predictions:
    print(f"Match {match}: {p['date']} - {p['team1']} vs {p['team2']} at {p['venue']}")
    print(f"If {p['team1']} bats first: {p['team1_bats_first'][0]} ({p['team1_bats_first'][1]*100:.2f}% confidence)")
    print(f"If {p['team2']} bats first: {p['team2_bats_first'][0]} ({p['team2_bats_first'][1]*100:.2f}% confidence)")
    print(f"Overall (average): {p['overall'][0]} ({p['overall'][1]*100:.2f}% confidence)\n")
    match = match + 1

Match 1: Dec 14 - Perth Scorchers vs Sydney Sixers at Perth Stadium
If Perth Scorchers bats first: Perth Scorchers (51.12% confidence)
If Sydney Sixers bats first: Perth Scorchers (58.55% confidence)
Overall (average): Perth Scorchers (54.83% confidence)

Match 2: Dec 15 - Melbourne Renegades vs Brisbane Heat at Simonds Stadium, South Geelong, Victoria
If Melbourne Renegades bats first: Brisbane Heat (75.62% confidence)
If Brisbane Heat bats first: Brisbane Heat (45.31% confidence)
Overall (average): Brisbane Heat (60.47% confidence)

Match 3: Dec 16 - Hobart Hurricanes vs Sydney Thunder at Bellerive Oval, Hobart
If Hobart Hurricanes bats first: Hobart Hurricanes (64.82% confidence)
If Sydney Thunder bats first: Hobart Hurricanes (63.17% confidence)
Overall (average): Hobart Hurricanes (64.00% confidence)

Match 4: Dec 17 - Sydney Sixers vs Adelaide Strikers at Sydney Cricket Ground
If Sydney Sixers bats first: Sydney Sixers (76.43% confidence)
If Adelaide Strikers bats first: Sydney S

In [25]:
import pandas as pd

results_data = []

for p in predictions:
    results_data.append({
        'Date': p['date'],
        'Home team': p['team1'],
        'Away team': p['team2'],
        'Venue': p['venue'],
        'Winner (Home team bats first)': p['team1_bats_first'][0],
        'Confidence (Home team bats first)': round(p['team1_bats_first'][1] * 100, 2),
        'Winner (Away team bats first)': p['team2_bats_first'][0],
        'Confidence (Away team bats first)': round(p['team2_bats_first'][1] * 100, 2),
        'Overall Winner': p['overall'][0],
        'Overall Confidence': round(p['overall'][1] * 100, 2)
    })

results_df = pd.DataFrame(results_data)

# Save results to an Excel file
excel_filename = 'bbl_winner_predictions.xlsx'
results_df.to_excel(excel_filename, index=False)

In [29]:
def predict_specific_match(predictor, team1, team2, venue, squads):
    team1_players = squads.get(team1, None)
    team2_players = squads.get(team2, None)

    result_team1_bats = predictor.predict_winner(team1, team2, venue, team1_players, team2_players)
    result_team2_bats = predictor.predict_winner(team2, team1, venue, team2_players, team1_players)

    team1_bats_confidence = result_team1_bats.get('confidence', 0)
    team2_bats_confidence = result_team2_bats.get('confidence', 0)

    avg_conf_team1 = (team1_bats_confidence + result_team2_bats.get('probabilities', {}).get(team1, 0)) / 2
    avg_conf_team2 = (result_team1_bats.get('probabilities', {}).get(team2, 0) + team2_bats_confidence) / 2

    overall_winner = team1 if avg_conf_team1 > avg_conf_team2 else team2
    overall_confidence = max(avg_conf_team1, avg_conf_team2)

    return {
        'team1_bats_first_winner': result_team1_bats['prediction'],
        'team1_bats_first_confidence': team1_bats_confidence,
        'team2_bats_first_winner': result_team2_bats['prediction'],
        'team2_bats_first_confidence': team2_bats_confidence,
        'overall_winner': overall_winner,
        'overall_confidence': overall_confidence
    }

# Example usage:
selected_team1 = input()
selected_team2 = input()
selected_venue = input()

results = predict_specific_match(predictor, selected_team1, selected_team2, selected_venue, squads)

print(f"Prediction if {selected_team1} bats first: {results['team1_bats_first_winner']} ({results['team1_bats_first_confidence']*100:.2f}% confidence)")
print(f"Prediction if {selected_team2} bats first: {results['team2_bats_first_winner']} ({results['team2_bats_first_confidence']*100:.2f}% confidence)")
print(f"Overall predicted winner: {results['overall_winner']} ({results['overall_confidence']*100:.2f}% confidence)")


Sydney Sixers
Hobart Hurricanes
Sydney Cricket Ground
Prediction if Sydney Sixers bats first: Sydney Sixers (73.90% confidence)
Prediction if Hobart Hurricanes bats first: Hobart Hurricanes (70.46% confidence)
Overall predicted winner: Sydney Sixers (47.77% confidence)
